In [1]:
import pandas as pd
import numpy as np


In [2]:
food_prices = pd.read_csv("wfp_food_prices_nga_hdx.csv", low_memory=False)

inflation=pd.read_csv("inflation_data.csv")


In [3]:
food_prices.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,KG,actual,Wholesale,NGN,175.92,1.5525
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,KG,actual,Wholesale,NGN,150.18,1.3254
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),KG,actual,Wholesale,NGN,358.7,3.1656
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,KG,actual,Wholesale,NGN,155.61,1.3733


In [4]:
food_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72466 entries, 0 to 72465
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       72466 non-null  object
 1   admin1     72466 non-null  object
 2   admin2     72466 non-null  object
 3   market     72466 non-null  object
 4   latitude   72466 non-null  object
 5   longitude  72466 non-null  object
 6   category   72466 non-null  object
 7   commodity  72466 non-null  object
 8   unit       72466 non-null  object
 9   priceflag  72466 non-null  object
 10  pricetype  72466 non-null  object
 11  currency   72466 non-null  object
 12  price      72466 non-null  object
 13  usdprice   72466 non-null  object
dtypes: object(14)
memory usage: 7.7+ MB


In [5]:
food_prices.isnull().sum()

date         0
admin1       0
admin2       0
market       0
latitude     0
longitude    0
category     0
commodity    0
unit         0
priceflag    0
pricetype    0
currency     0
price        0
usdprice     0
dtype: int64

In [6]:
food_prices.duplicated().sum()

0

In [7]:
food_prices.shape

(72466, 14)

In [8]:
food_prices['priceflag'].value_counts()

priceflag
actual              51249
aggregate           21216
#item+price+flag        1
Name: count, dtype: int64

## Renaming the columns 

In [9]:
def preprocess_dataframe(food_prices):
    # Rename columns
    food_prices.rename(columns={'admin1': 'State', 'admin2': 'Local_Government', 'market': 'Market_Name'}, inplace=True)
    food_prices = food_prices.rename(columns={'admin2' : 'local_gov_area'})

    # Drop the first row
    food_prices = food_prices.iloc[1:]

    # Convert the 'Date' column to datetime type
    food_prices['date'] = pd.to_datetime(food_prices['date'], format='%Y-%m-%d')
    food_prices['year'] = food_prices['date'].dt.year
    food_prices['month'] = food_prices['date'].dt.month

    # Create a mapping dictionary for month names
    month_mapping = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                     7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

    # Use the map function to replace the values in the 'month' column
    food_prices['month'] = food_prices['month'].map(month_mapping)

   
    # Reset the index
    food_prices.reset_index(drop=True, inplace=True)

    # Display the updated DataFrame
    return food_prices

In [10]:
df_food = preprocess_dataframe(food_prices)
df_food.head(2)

,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,KG,actual,Wholesale,NGN,175.92,1.5525,2002,January
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,KG,actual,Wholesale,NGN,150.18,1.3254,2002,January


In [11]:
df_food.shape

(72465, 16)

In [12]:
df_food['unit'].value_counts()

unit
100 KG        16119
KG            15292
1.3 KG         5952
L              4804
1.4 KG         4227
1.2 KG         3545
2.5 KG         2940
750 ML         2598
0.5 KG         2115
400 G          2111
50 KG          1689
20 G           1440
30 pcs         1413
2.1 KG         1403
300 G          1170
250 G          1109
Unit            947
3.1 KG          708
1.1 KG          592
500 G           588
3.4 KG          578
1.5 KG          574
100 Tubers      472
100 L            79
Name: count, dtype: int64

In [13]:
def convert_to_kg(row):
    unit = row['unit']
    price = row['price']

    # Exclude these units from conversion
    exclude_units = ['30 pcs', '100 Tubers']

    # If the unit is not excluded and contains 'KG', extract the numeric value
    if unit not in exclude_units and 'KG' in unit.upper():
        numeric_value = ''.join(filter(str.isdigit, unit))

        # If numeric_value is not an empty string, convert it to a float
        if numeric_value:
            numeric_value = float(numeric_value)

            # If numeric_value is greater than 0, calculate the price per kg and update the unit
            if numeric_value > 0:
                # Ensure price is not an empty string before converting to float
                if price:
                    price = float(price)
                    price /= numeric_value
                unit = '1KG'
    elif unit in ['100 L', '20 G', '750 ML', '400 G', '300 G', '250 G', '500 G']:
        # Convert the 'price' column to a numerical data type
        price = pd.to_numeric(price)

        # Calculate the equivalent price for 1 KG
        if unit == '100 L':
            price_per_kg = price / 100  # Convert 100 L to 1 KG
        elif unit == '20 G':
            price_per_kg = price * 50   # Convert 20 G to 1 KG
        elif unit == '750 ML':
            price_per_kg = price * 1.3333  # Convert 750 ML to 1 KG (approximately)
        elif unit == '400 G':
            price_per_kg = price * 2.5  # Convert 400 G to 1 KG
        elif unit == '300 G':
            price_per_kg = price * 3.3333  # Convert 300 G to 1 KG (approximately)
        elif unit == '250 G':
            price_per_kg = price * 4.0  # Convert 250 G to 1 KG
        elif unit == '500 G':
            price_per_kg = price * 2  # Convert 500 G to 1 KG


        unit = '1KG'  # Update the unit to '1 KG'
        price = price_per_kg

    return price, unit

# Apply the function to calculate price per 1 KG and update the unit
df_food[['price', 'unit']] = df_food.apply(convert_to_kg, axis=1, result_type='expand')

# Ensure all 'KG' units are represented as '1KG'
df_food['unit'] = df_food['unit'].replace('KG', '1KG')

# Display the first few rows of the dataframe
df_food.head()


,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January


In [14]:
# Define a function to map months to seasons
def map_to_season(month):
    if month in ['December', 'January', 'February']:
        return 'Dry Season'
    elif month in ['March', 'April', 'May']:
        return 'Early Rainy Season'
    elif month in ['June', 'July', 'August', 'September']:
        return 'Peak Rainy Season'
    else:
        return 'Late Rainy Season'

# Apply the function to create the 'season' column
df_food['season'] = df_food['month'].apply(map_to_season)
df_food.head()

,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,season
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January,Dry Season
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January,Dry Season
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January,Dry Season
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January,Dry Season
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January,Dry Season


In [15]:
df_food['unit'].unique()

array(['1KG', 'Unit', 'L', '30 pcs', '100 Tubers'], dtype=object)

In [16]:
inflation=pd.read_csv("inflation_data.csv")
inflation.head()

,DATE,Inflation
0,01-01-1960,5.444327
1,01-01-1961,6.279147
2,01-01-1962,5.265632
3,01-01-1963,-2.694655
4,01-01-1964,0.856793


In [17]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DATE       64 non-null     object 
 1   Inflation  64 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


In [18]:
inflation['DATE'].unique()

array(['01-01-1960', '01-01-1961', '01-01-1962', '01-01-1963',
       '01-01-1964', '01-01-1965', '01-01-1966', '01-01-1967',
       '01-01-1968', '01-01-1969', '01-01-1970', '01-01-1971',
       '01-01-1972', '01-01-1973', '01-01-1974', '01-01-1975',
       '01-01-1976', '01-01-1977', '01-01-1978', '01-01-1979',
       '01-01-1980', '01-01-1981', '01-01-1982', '01-01-1983',
       '01-01-1984', '01-01-1985', '01-01-1986', '01-01-1987',
       '01-01-1988', '01-01-1989', '01-01-1990', '01-01-1991',
       '01-01-1992', '01-01-1993', '01-01-1994', '01-01-1995',
       '01-01-1996', '01-01-1997', '01-01-1998', '01-01-1999',
       '01-01-2000', '01-01-2001', '01-01-2002', '01-01-2003',
       '01-01-2004', '01-01-2005', '01-01-2006', '01-01-2007',
       '01-01-2008', '01-01-2009', '01-01-2010', '01-01-2011',
       '01-01-2012', '01-01-2013', '01-01-2014', '01-01-2015',
       '01-01-2016', '01-01-2017', '01-01-2018', '01-01-2019',
       '01-01-2020', '01-01-2021', '01-01-2022', '01-01

In [19]:
inflation['DATE'] = pd.to_datetime(inflation['DATE'], format='%d-%m-%Y')
inflation['year'] = inflation['DATE'].dt.year

# Filter data from 2002 to the last year in the DataFrame
df_inf= inflation[inflation['year'] >= 2002]

df_inf.drop(columns=['DATE'], inplace=True)
df_inf.head()

C:\Users\abrah\AppData\Local\Temp\ipykernel_24560\2259494106.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf.drop(columns=['DATE'], inplace=True)


,Inflation,year
42,12.876579,2002
43,14.031784,2003
44,14.998034,2004
45,17.863493,2005
46,8.225222,2006


In [20]:
merged_df = df_food.merge(df_inf, on=['year'], how='left')
merged_df.head()

,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,season,Inflation
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January,Dry Season,12.876579
1,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Millet,1KG,actual,Wholesale,NGN,150.18,1.3254,2002,January,Dry Season,12.876579
2,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Rice (imported),1KG,actual,Wholesale,NGN,358.7,3.1656,2002,January,Dry Season,12.876579
3,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,cereals and tubers,Sorghum,1KG,actual,Wholesale,NGN,155.61,1.3733,2002,January,Dry Season,12.876579
4,2002-01-15,Katsina,Jibia,Jibia (CBM),13.08,7.24,pulses and nuts,Beans (niebe),1KG,actual,Wholesale,NGN,196.87,1.7374,2002,January,Dry Season,12.876579


In [21]:
print(merged_df.columns)
print(merged_df.shape)
print(merged_df.info())

Index(['date', 'State', 'Local_Government', 'Market_Name', 'latitude',
       'longitude', 'category', 'commodity', 'unit', 'priceflag', 'pricetype',
       'currency', 'price', 'usdprice', 'year', 'month', 'season',
       'Inflation'],
      dtype='object')
(72465, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72465 entries, 0 to 72464
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              72465 non-null  datetime64[ns]
 1   State             72465 non-null  object        
 2   Local_Government  72465 non-null  object        
 3   Market_Name       72465 non-null  object        
 4   latitude          72465 non-null  object        
 5   longitude         72465 non-null  object        
 6   category          72465 non-null  object        
 7   commodity         72465 non-null  object        
 8   unit              72465 non-null  object        
 9   priceflag         72

In [22]:
# converts the specified columns to their appropriate data types
merged_df['latitude'] = merged_df['latitude'].astype(float)
merged_df['longitude'] = merged_df['longitude'].astype(float)
merged_df['price'] = merged_df['price'].astype(float)
merged_df['usdprice'] = merged_df['usdprice'].astype(float)
#merged_df['Inflation'] = merged_df['Inflation'].astype(float)
merged_df['date'] = pd.to_datetime(merged_df['date'])

In [23]:
# Compute the correlation matrix
corr_matrix = merged_df[['latitude', 'longitude', 'price', 'usdprice', 'year','Inflation']].corr()

# Print the correlation matrix
corr_matrix


,latitude,longitude,price,usdprice,year,Inflation
latitude,1.000000,0.428606,-0.010542,-0.244856,-0.044257,-0.000852
longitude,0.428606,1.000000,0.025909,-0.307443,0.226641,0.145048
price,-0.010542,0.025909,1.000000,0.191303,0.090594,0.095727
usdprice,-0.244856,-0.307443,0.191303,1.000000,-0.098244,-0.029021
year,-0.044257,0.226641,0.090594,-0.098244,1.000000,0.600313
Inflation,-0.000852,0.145048,0.095727,-0.029021,0.600313,1.000000


In [24]:
merged_df.to_csv('preprocessed_food_inflation.csv', index=False)